In [ ]:
%%capture
!pip install datasets llama-index==0.10.37 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-cohere==0.2.0

Note: you should install the following packages to your environment:

`pip install datasets`

`pip install llama-index-embeddings-fastembed`

`pip install llama-index-llms-mistralai`

In [ ]:
import os
import sys
from dotenv import load_dotenv
from getpass import getpass
import nest_asyncio
from IPython.display import Markdown, display

nest_asyncio.apply()

load_dotenv()

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, create_index

In [ ]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [ ]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [ ]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [ ]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [ ]:
from llama_index.core.settings import Settings
from llama_index.core import StorageContext

from utils import setup_llm, setup_embed_model, setup_vector_store, create_index

setup_llm(provider="cohere", model="command-r-plus", api_key=CO_API_KEY)

setup_embed_model(provider="openai", api_key=OPENAI_API_KEY)

COLLECTION_NAME = "it_can_be_done"

vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = create_index(from_where="vector_store", vector_store=vector_store, storage_context=storage_context)

# Default Prompt Templates

In [ ]:
from utils import display_prompt_dict, create_query_engine

In [ ]:
query_engine = create_query_engine(
    index=index,
    similarity_top_k=3, 
    mode="query",
    return_sources=True
    )

display_prompt_dict(query_engine.get_prompts())

In [ ]:
from llama_index.core.prompts import PromptTemplate

custom_prompt = """You are an assistant for question-answering tasks related to \
motivational poetry. Your must reponse with an original Haiku style poem.

Use the following pieces of retrieved context to answer the user's query:

---------------------\n
{context_str}\n
---------------------\n

Query: {query_str}
"""

custom_prompt_template = PromptTemplate(custom_prompt)

In [ ]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": custom_prompt_template}
)

In [ ]:
display_prompt_dict(query_engine.get_prompts())

In [ ]:
from utils import create_query_pipeline

from llama_index.core.query_pipeline import InputComponent

input_component = InputComponent()

chain = [input_component, query_engine]

query_pipeline = create_query_pipeline(chain)

In [ ]:
response = query_pipeline.run(input="If you keep your head when all around you are losing their cool and blaming it on you")

In [ ]:
print(response)

# Response Synthesizers

The Llama Index [documentation](https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/) has a lot of detail regarding each of the response sythensizers. 

In [ ]:
from llama_index.core.response_synthesizers import ResponseMode

In [ ]:
dir(ResponseMode)

# Response modes

In LlamaIndex, [response modes](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes/) are used to determine how the system should process and return the results of a query.  Each response mode is designed to handle different types of queries and use cases, providing flexibility and customization in how you interact with your data.



#### ⚗️ Refine 

Refine is an iterative method to generate a response. 

Initially, we use the context in the first node and the query to create a basic answer. Then, we refine this answer by inputting it, along with the query and context of the second node, into a "refine prompt" to generate an improved answer. 

This refinement process continues through N-1 nodes, with N being the total number of nodes. It makes a separate LLM call per Node/retrieved chunk. This mode is good for generating more detailed answers.

#### 🤏 Compact

Compact and refine mode first combine text chunks into larger consolidated chunks that more fully utilize the available context window, then refine answers across them. This mode is faster than refine since we make fewer calls to the LLM. 

This mode is useful when you want to reduce the number of LLM calls while still refining the answer.


#### 📝 Simple summarize

Merge all text chunks into one and make a large language model call. The call will fail if the merged text chunk exceeds the context window size.

It's good for quick summarization purposes, but may lose detail due to truncation.

#### 🌴 Tree summarize

Construct a tree index for the candidate nodes in a bottom-up manner then use a summary prompt based on the query. Return the root node as the final response. When this mode is set, the system is instructed to iterate through many, if not all, documents in order to synthesize an answer, which can lead to better summarization results. 

This mode is particularly useful for summarization queries, where the goal is to provide a comprehensive summary of a collection of text or a specific topic.

#### 🤖 Generation

Ignore context, just use LLM to generate a response.

#### ❌ No text

This mode only runs the retriever to fetch the nodes that would have been sent to the LLM, without actually synthesizing a final response. The nodes can then be inspected by checking `response.source_nodes`.

#### 📏 Accumulate

This mode applies the query to each text chunk while accumulating the responses into an array. It returns a concatenated string of all responses. 

This mode is good for when you need to run the same query separately against each text chunk.

#### Compact accumulate

In the compact and accumulate mode, text chunks are combined into larger chunks to utilize the context window better. Answers are then accumulated for each chunk and returned as a concatenation. This mode is faster than accumulate as it reduces calls to the LLM.

In [ ]:
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = create_query_engine(
    index,
    mode="query",
    response_synthesizer = response_synthesizer
    )

input_component = InputComponent()

chain = [input_component, query_engine, Settings.llm,]

query_pipeline = create_query_pipeline(chain)

query_pipeline.run(input="What do the poems teach about one should think about success and failure?")